# Pre generated model: Rock-salt

Here we run "step 4" separately to define a common object for all materials of the same type.

This is useful because each time you run QSYMM to define the kp model, its numerical process may lead Hamiltonian families that are equivalent, but with different numerical prefactors or different order of the $c_n$ coefficients. Therefore, here define the model and store it in a file to be read by the examples that uses this crystal symmetry.

In [1]:
import numpy as np
import pydft2kp as dft2kp
from qsymm.hamiltonian_generator import hamiltonian_from_family
import sympy
import pickle

In [6]:
# step 4: builds optimal model with qsymm
# s0, sx, sy, sz: Pauli matrices
from pydft2kp.constants import s0, sx, sy, sz
from scipy.linalg import expm, block_diag

# build symmetry operations of D3d
# consider also T = time-reversal symmetry

# ------------------------------------
sigma = (sx + sy + sz)/np.sqrt(3)
U = np.kron(s0, expm(-1j*(np.pi/3)*sigma))
C3 = dft2kp.rotation(1/3, [1,1,1], U=U)

sigma = (-sx + sy)/np.sqrt(2)
U = np.kron(sz, expm(-1j*(np.pi/2)*sigma))
C2 = dft2kp.rotation(1/2, [-1,1,0], U=U)

U = np.kron(sz, s0)
Inv = dft2kp.inversion(3, U=U)

U = np.kron(s0, 1j*sy)
TRS = dft2kp.time_reversal(3, U=U)
# ------------------------------------

symms = [C2, C3, Inv, TRS]
qs = dft2kp.qsymm(symms, dim=3, total_power=2);

Matrix([
[c0 + c4*k_x**2 + c4*k_y**2 + c4*k_z**2 + c6*k_x*k_y + c6*k_x*k_z + c6*k_y*k_z,                                                                             0,                             -c2*k_x + c2*k_y + I*c3*k_x + I*c3*k_y + I*c3*k_z,                                           -I*c2*k_x - c2*k_y + c2*k_z*(1 + I)],
[                                                                            0, c0 + c4*k_x**2 + c4*k_y**2 + c4*k_z**2 + c6*k_x*k_y + c6*k_x*k_z + c6*k_y*k_z,                                            I*c2*k_x - c2*k_y + c2*k_z*(1 - I),                              c2*k_x - c2*k_y + I*c3*k_x + I*c3*k_y + I*c3*k_z],
[                            -c2*k_x + c2*k_y - I*c3*k_x - I*c3*k_y - I*c3*k_z,                                           -I*c2*k_x - c2*k_y + c2*k_z*(1 + I), c1 + c5*k_x**2 + c5*k_y**2 + c5*k_z**2 + c7*k_x*k_y + c7*k_x*k_z + c7*k_y*k_z,                                                                             0],
[                                    

In [7]:
# save symms and qs object
with open('rocksalt.pickle', 'wb') as fp:
  pickle.dump(symms, fp)
  pickle.dump(qs, fp)

# Organazing H to print as latex for paper

In [8]:
# print model in Latex format
H = hamiltonian_from_family(qs.model, nsimplify=True)

cs = sympy.symbols('c:17')
kx, ky, kz = sympy.symbols('k_x, k_y, k_z')
k2, kp, km = sympy.symbols('k^2, k_+, k_-')
kxy2 = sympy.symbols('k_{\parallel}^2') # = kx**2 + ky**2
Kbig = sympy.symbols('\hat{K}') # = kx**2 - ky**2
# kp = kx+i.ky
# km = kx-i.ky
# kx = (kp+km)/2
# ky = (kp-km)/2i

H = H.applyfunc(lambda ij: sympy.collect(ij, cs))

H = sympy.simplify(H)
H = sympy.simplify(H.subs({kx**2 + ky**2 + kz**2: k2}))
H = sympy.simplify(H.subs({kx**2 + ky**2: kxy2}))
H = sympy.simplify(H.subs({kx+sympy.I*ky: kp, 
                           kx-sympy.I*ky: km,
                           sympy.I*kx-ky: sympy.I*kp, 
                           sympy.I*kx+ky: sympy.I*km,
                           }))

H = sympy.simplify(H.subs({kx**2 - ky**2: Kbig}))

H

Matrix([
[c0 + c4*k^2 + c6*(k_x*k_y + k_x*k_z + k_y*k_z),                                              0,       -c2*(k_x - k_y) + I*c3*(k_x + k_y + k_z),                      c2*(-I*k_- + k_z*(1 + I))],
[                                             0, c0 + c4*k^2 + c6*(k_x*k_y + k_x*k_z + k_y*k_z),                       c2*(I*k_+ + k_z*(1 - I)),        c2*(k_x - k_y) + I*c3*(k_x + k_y + k_z)],
[      -c2*(k_x - k_y) - I*c3*(k_x + k_y + k_z),                      c2*(-I*k_- + k_z*(1 + I)), c1 + c5*k^2 + c7*(k_x*k_y + k_x*k_z + k_y*k_z),                                              0],
[                      c2*(I*k_+ + k_z*(1 - I)),        c2*(k_x - k_y) - I*c3*(k_x + k_y + k_z),                                              0, c1 + c5*k^2 + c7*(k_x*k_y + k_x*k_z + k_y*k_z)]])

In [9]:
print(sympy.latex(H))

\left[\begin{matrix}c_{0} + c_{4} k^{2} + c_{6} \left(k_{x} k_{y} + k_{x} k_{z} + k_{y} k_{z}\right) & 0 & - c_{2} \left(k_{x} - k_{y}\right) + i c_{3} \left(k_{x} + k_{y} + k_{z}\right) & c_{2} \left(- i k_{-} + k_{z} \left(1 + i\right)\right)\\0 & c_{0} + c_{4} k^{2} + c_{6} \left(k_{x} k_{y} + k_{x} k_{z} + k_{y} k_{z}\right) & c_{2} \left(i k_{+} + k_{z} \left(1 - i\right)\right) & c_{2} \left(k_{x} - k_{y}\right) + i c_{3} \left(k_{x} + k_{y} + k_{z}\right)\\- c_{2} \left(k_{x} - k_{y}\right) - i c_{3} \left(k_{x} + k_{y} + k_{z}\right) & c_{2} \left(- i k_{-} + k_{z} \left(1 + i\right)\right) & c_{1} + c_{5} k^{2} + c_{7} \left(k_{x} k_{y} + k_{x} k_{z} + k_{y} k_{z}\right) & 0\\c_{2} \left(i k_{+} + k_{z} \left(1 - i\right)\right) & c_{2} \left(k_{x} - k_{y}\right) - i c_{3} \left(k_{x} + k_{y} + k_{z}\right) & 0 & c_{1} + c_{5} k^{2} + c_{7} \left(k_{x} k_{y} + k_{x} k_{z} + k_{y} k_{z}\right)\end{matrix}\right]
